In [ ]:
import pandas as pd
import glob
import os

# ------------------------------------------------------------------------------
# LÓGICA DE EXTRAÇÃO DE ESTADO (COPIADA DO 'estado_extractor.py')
# ------------------------------------------------------------------------------

def obter_mapeamento_estados():
    """
    Retorna o dicionário de mapeamento dos códigos de duas letras 
    (presentes no nome do arquivo) para os nomes completos dos estados/entidades.
    """
    return {
        'al': 'Alagoas',
        'am': 'Amazonas',
        'ap': 'Amapá',
        'ba': 'Bahia',
        'ce': 'Ceará',
        'df': 'Distrito Federal',
        'es': 'Espírito Santo',
        'go': 'Goiás',
        'hm': 'Hemominas',
        'ma': 'Maranhão',
        'mg': 'Minas Gerais', # Adicionado para teste/exemplo
        'rj': 'Rio de Janeiro', # Adicionado para teste/exemplo
        # Adicione outros estados/entidades conforme necessário:
        # 'sp': 'São Paulo',
        # ...
    }

def extrair_estado_do_nome_arquivo(nome_arquivo):
    """
    Extrai o nome do estado a partir do nome do arquivo, assumindo o formato 'hemoprod_XX_...'.
    
    IMPORTANTE: Este nome de arquivo DEVE VIR SEM A EXTENSÃO (.xlsx)
    """
    mapeamento = obter_mapeamento_estados()
    
    # 1. Tenta extrair o código de duas letras após 'hemoprod_'
    try:
        # Exemplo: 'hemoprod_al_dados' -> 'al_dados'
        # Se 'hemoprod_' não estiver presente, pode dar IndexError
        parte_codigo = nome_arquivo.split('hemoprod_', 1)[1]
        
        # Exemplo: 'al_dados' -> 'al'
        codigo = parte_codigo.split('_', 1)[0].lower()
        
        # 2. Busca o código no mapeamento
        estado = mapeamento.get(codigo)
        
        if estado:
            return estado
        else:
            return f'Estado Desconhecido (Código: {codigo})'
            
    except IndexError:
        # Se o formato 'hemoprod_...' não for encontrado
        return 'Estado Desconhecido (Formato Inválido)'

# ------------------------------------------------------------------------------
# SCRIPT PRINCIPAL DE PROCESSAMENTO
# ------------------------------------------------------------------------------

# 1. Defina o caminho para os seus arquivos XLSX
# Altere 'caminho/para/seus/arquivos' para o diretório onde os arquivos .xlsx estão.
path = 'dados_processados' 
all_files = glob.glob(os.path.join(path, "*.xlsx"))

# Lista para armazenar os DataFrames de cada arquivo
lista_dfs = []

# 2. Leia cada arquivo XLSX e adicione à lista
print("Iniciando a leitura dos arquivos XLSX...")
for filename in all_files:
    try:
        df = pd.read_excel(filename, sheet_name=0) 
        
        # --- AJUSTE AQUI: Remove a extensão antes de extrair o estado ---
        nome_completo = os.path.basename(filename)
        # nome_base: 'hemoprod_al_dados' (sem .xlsx)
        nome_base, _ = os.path.splitext(nome_completo)
        
        # Adiciona a coluna com o nome do arquivo de origem (com extensão)
        df['origem_arquivo'] = nome_completo
        
        # NOVO: Adiciona a coluna 'estado' usando a função de extração
        # Passa o nome_base SEM a extensão para a função
        estado = extrair_estado_do_nome_arquivo(nome_base)
        df['estado'] = estado
        
        lista_dfs.append(df)
        print(f"Arquivo lido com sucesso: {filename}. Estado atribuído: {estado}")
        
    except Exception as e:
        print(f"ERRO ao ler o arquivo {filename}: {e}")
        
# 3. Concatene todos os DataFrames em um único DataFrame
if lista_dfs:
    # Concatena todos os DataFrames
    df_final = pd.concat(lista_dfs, axis=0, ignore_index=True)
    print("\nTodos os arquivos foram concatenados em um único DataFrame.")
    print(f"Shape do DataFrame final: {df_final.shape}")

    # --- VERIFICAÇÃO DO CAMPO 'estado' (HEAD) ---
    print("\n--- Head da Coluna 'estado' para verificação ---")
    print(df_final['estado'].head(10))
    print("-------------------------------------------------")
    # ---------------------------------------------------

    # -------------------------------------------------------------
    # TRATAMENTO DE COLUNAS DE DATA (SOLUÇÃO PARA O ArrowTypeError)
    # -------------------------------------------------------------
    colunas_datas = ['data_inicio', 'data_ultima_acao']
    
    for coluna in colunas_datas:
        if coluna in df_final.columns:
            print(f"Tentando converter a coluna '{coluna}' para o tipo datetime...")
            df_final[coluna] = pd.to_datetime(df_final[coluna], errors='coerce')
            print(f"Conversão de '{coluna}' concluída. Novo tipo: {df_final[coluna].dtype}")

    # -------------------------------------------------------------
    # TRATAMENTO: COLUNAS STRING (SOLUÇÃO PARA TIPOS MISTOS)
    # -------------------------------------------------------------
    colunas_strings = ['ip', 'cnpj', 'cnes', 'hemoprod_1_observacoes', 'hemoprod_3_observacoes', 'estado'] 
    
    for coluna in colunas_strings:
        if coluna in df_final.columns:
            print(f"Tentando converter a coluna '{coluna}' para o tipo string...")
            df_final[coluna] = df_final[coluna].astype(str)
            print(f"Conversão de '{coluna}' concluída. Novo tipo: {df_final[coluna].dtype}")

    # -------------------------------------------------------------

    # 4. Salve o DataFrame final como um arquivo Parquet
    nome_arquivo_parquet = 'dados_processados/hemoprod_nacional.parquet'
    
    df_final.to_parquet(nome_arquivo_parquet, engine='pyarrow', index=False)
    
    print(f"\nDataFrame salvo com sucesso como '{nome_arquivo_parquet}'")
else:
    print("\nNenhum arquivo XLSX foi encontrado ou lido com sucesso. O Parquet não foi criado.")

Iniciando a leitura dos arquivos XLSX...
Arquivo lido com sucesso: dados_processados\hemoprod_al.xlsx. Estado atribuído: Alagoas
Arquivo lido com sucesso: dados_processados\hemoprod_am.xlsx. Estado atribuído: Amazonas
Arquivo lido com sucesso: dados_processados\hemoprod_ap.xlsx. Estado atribuído: Amapá
Arquivo lido com sucesso: dados_processados\hemoprod_ba.xlsx. Estado atribuído: Bahia
Arquivo lido com sucesso: dados_processados\hemoprod_ce.xlsx. Estado atribuído: Ceará
Arquivo lido com sucesso: dados_processados\hemoprod_df.xlsx. Estado atribuído: Distrito Federal
Arquivo lido com sucesso: dados_processados\hemoprod_es.xlsx. Estado atribuído: Espírito Santo
Arquivo lido com sucesso: dados_processados\hemoprod_go.xlsx. Estado atribuído: Goiás
Arquivo lido com sucesso: dados_processados\hemoprod_hm.xlsx. Estado atribuído: Hemominas
Arquivo lido com sucesso: dados_processados\hemoprod_ma.xlsx. Estado atribuído: Maranhão

Todos os arquivos foram concatenados em um único DataFrame.
Shape 

C:\Users\victo\AppData\Local\Temp\ipykernel_28980\2766181610.py:101: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat(lista_dfs, axis=0, ignore_index=True)
